# Change the ecoinvent version of the mapping file

In [1]:
import pandas as pd
import bw2data as bd
from mescal import *

In [2]:
esm_location = 'CA-QC' # can be 'CH' or 'CA-QC'
ei_version_from = '3.8'
ei_version_to = '3.9.1'

In [3]:
bd.projects.set_current(f"ecoinvent{ei_version_to}A")

In [4]:
name_comp_db = f'ecoinvent_cutoff_{ei_version_from}_remind_SSP2-Base_2020_comp_{esm_location}'

In [5]:
mapping = pd.read_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_from}_linked.csv")

In [6]:
updated_mapping, unit_changes = change_ecoinvent_version_mapping(mapping, ei_version_from, ei_version_to, name_comp_db)

BATTERY Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO
New: battery, Li-ion, LiMn2O4, rechargeable, prismatic - market for battery, Li-ion, LiMn2O4, rechargeable, prismatic - GLO
CEMENT_PROD Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - RoW
New: cement, CEM II/B - cement production, CEM II/B - RoW
CEMENT_PROD_HP Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - RoW
New: cement, CEM II/B - cement production, CEM II/B - RoW
ELEC_STO Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO
New: battery, Li-ion, LiMn2O4, rechargeable, prismatic - market for battery, Li-ion, LiMn2O4, rechargeable, prismatic - GLO
NG_COMP_HE Operation
Old: natural gas, high pressure - market for natural gas, high pressure - CA-QC
New: natural gas, high pr

In [7]:
updated_mapping.head()

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020


In [8]:
updated_mapping.Database.unique()

array(['ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020',
       'urban delivery_truck', 'regional delivery_truck'], dtype=object)

In [9]:
base_db = Database(db_names=list(updated_mapping.Database.unique()))
unlinked = base_db.test_mapping_file(updated_mapping)

Getting activity data


100%|██████████| 28551/28551 [00:00<00:00, 62244.18it/s]


Adding exchange data to activities


100%|██████████| 838277/838277 [00:35<00:00, 23436.23it/s]


Filling out exchange data


...

Adding exchange data to activities


100%|██████████| 30702/30702 [00:02<00:00, 13289.75it/s]


Filling out exchange data


100%|██████████| 935/935 [00:00<00:00, 1847.51it/s]


Loaded regional delivery_truck from brightway!
Mapping successfully linked to the database


In [10]:
if len(unlinked) == 0:
    updated_mapping.to_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_to}.csv", index=False)
else:
    print(f"Unlinked exchanges: {unlinked}")

# Adapt the unit conversion file corresponding to the new mapping file

In [18]:
unit_conversion = pd.read_csv(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_from}.csv")

In [19]:
unit_changes

[[('OCGT_LARGE', 'Operation'),
  ('electricity, medium voltage',
   'natural gas, burned in gas turbine, for compressor station',
   'CA-QC'),
  'kilowatt hour',
  'megajoule'],
 [('OCGT_SMALL', 'Operation'),
  ('electricity, medium voltage',
   'natural gas, burned in gas turbine, for compressor station',
   'CA-QC'),
  'kilowatt hour',
  'megajoule']]

In [20]:
# Add new unit conversion factors
new_unit_conversion_factors = {
    ('OCGT_LARGE', 'Operation'): ['= 3.6 * 2.5', '3.6 MJ of heat per kWh, 2.5 kWh NG per kWh electricity'],
    ('OCGT_SMALL', 'Operation'): ['= 3.6 * 2.5', '3.6 MJ of heat per kWh, 2.5 kWh NG per kWh electricity'],
}

In [21]:
unit_conversion = update_unit_conversion_file(unit_conversion, unit_changes, new_unit_conversion_factors)

In [22]:
unit_conversion.sort_values(by=['Name', 'Type']).to_csv(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_to}.csv", index=False)